<a href="https://colab.research.google.com/github/VishnuGupta5883/appliedai/blob/master/SQL_Assignment_on_IMDB_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import sqlite3
conn = sqlite3.connect("/content/drive/My Drive/Colab Notebooks/IMDB.db")
#cur = conn.cursor()
#cur.execute("select * from Person limit 5;")
#results = cur.fetchall()
#print(results)
#result = pd.read_sql_query("SELECT * FROM M_CAST LIMIT 10 ", conn)
result = pd.read_sql_query("SELECT * FROM MOVIE LIMIT 10 ", conn)
result


,index,MID,title,year,rating,num_votes
0,0,tt2388771,Mowgli,2018,6.6,21967
1,1,tt5164214,Ocean's Eight,2018,6.2,110861
2,2,tt1365519,Tomb Raider,2018,6.4,142585
3,3,tt0848228,The Avengers,2012,8.1,1137529
4,4,tt8239946,Tumbbad,2018,8.5,7483
5,5,tt7027278,Kedarnath,2018,5.5,1970
6,6,tt3498820,Captain America: Civil War,2016,7.8,536641
7,7,tt8108198,Andhadhun,2018,9.0,18160
8,8,tt3741834,Lion,2016,8.1,170216
9,9,tt6747420,Rajma Chawal,2018,5.7,681


1. List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that
the genre is 'Comedy’ and year is a leap year) Your query should return director name, the
movie name, and the year.

In [0]:
res = pd.read_sql_query(
                        "SELECT  \
                          DISTINCT TRIM(P.Name) As Director_Name\
                        ,M.title As Movie_Name \
                        ,M.year AS Year \
                        FROM M_GENRE AS M_G \
                        LEFT JOIN GENRE AS G \
                        ON M_G.GID = G.GID \
                        ,    Movie As M \
                        LEFT JOIN Person As P \
                        ,         M_Director As MD \
                        ON M.MID = MD.MID AND P.PID = MD.PID \
                        WHERE (Trim(M.year) %4 ==0) \
                        AND M.MID = M_G.MID AND Trim(G.NAME) LIKE '%COMEDY%' ", conn)
res

,Director_Name,Movie_Name,Year
0,Milap Zaveri,Mastizaade,2016
1,Danny Leiner,Harold & Kumar Go to White Castle,2004
2,Anurag Kashyap,Gangs of Wasseypur,2012
3,Frank Coraci,Around the World in 80 Days,2004
4,Griffin Dunne,The Accidental Husband,2008
...,...,...,...
241,Siddharth Anand Kumar,Let's Enjoy,2004
242,Amma Rajasekhar,Sathyam,2008
243,Oliver Paulus,Tandoori Love,2008
244,Raja Chanda,Le Halua Le,2012


2. List the names of all the actors who played in the movie 'Anand' (1971)

In [0]:
res1 = pd.read_sql_query(
                        "SELECT  \
                        P.Name As Actor_Name\
                        FROM Movie As M \
                        LEFT JOIN Person As P \
                        ,         M_Cast As C \
                        ON TRIM(M.MID) = TRIM(C.MID) \
                        AND TRIM(P.PID) = TRIM(C.PID) \
                        WHERE M.title LIKE 'Anand' ", conn)
res1

,Actor_Name
0,Amitabh Bachchan
1,Rajesh Khanna
2,Sumita Sanyal
3,Ramesh Deo
4,Seema Deo
5,Asit Kumar Sen
6,Dev Kishan
7,Atam Prakash
8,Lalita Kumari
9,Savita


3. List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970
and > 1990.)

In [0]:
res1 = pd.read_sql_query(
                        "SELECT  \
                        TRIM(P.Name) As Actor_Name \
                        FROM Movie As M \
                        LEFT JOIN Person As P \
                        ,         M_Cast As C \
                        ON M.MID = C.MID \
                        AND P.PID = Trim(C.PID) \
                        WHERE CAST(SUBSTR(M.YEAR,-4) AS INT) < 1970 \
                        AND P.PID IN (\
                                        SELECT  \
                                        TRIM(C2.PID)  \
                                        FROM Movie As M2 \
                                        LEFT JOIN M_Cast As C2 \
                                            ON M2.MID = C2.MID \
                                        WHERE CAST(SUBSTR(M2.YEAR,-4) AS INT) > 1990 ) \
                  ", conn)
res1

,Actor_Name
0,Waheeda Rehman
1,Johnny Walker
2,Mehmood
3,Ratna
4,Mehmood
...,...
1611,Rajan Haksar
1612,Anoop Kumar
1613,Shammi Kapoor
1614,Rekha


4. List all directors who directed 10 movies or more, in descending order of the number of
movies they directed. Return the directors' names and the number of movies each of them
directed.

In [0]:
res = pd.read_sql_query(
                        "SELECT  \
                              TRIM(P.Name)       As Director_Name \
                              ,COUNT(M.MID)       As Movie_COUNT \
                        FROM Movie As M \
                        LEFT JOIN Person As P \
                        ,         M_Director As MD \
                        ON M.MID = MD.MID \
                        AND P.PID = MD.PID \
                        GROUP BY TRIM(P.NAME) \
                        HAVING COUNT(M.MID) >= 10 \
                        ORDER BY COUNT(M.MID) DESC ", conn)
res

,Director_Name,Movie_COUNT
0,David Dhawan,78
1,Mahesh Bhatt,72
2,Ram Gopal Varma,60
3,Vikram Bhatt,58
4,Hrishikesh Mukherjee,54
...,...,...
151,Siddharth Anand,10
152,Subhash Kapoor,10
153,Subodh Mukherji,10
154,Sujoy Ghosh,10


5.

a. For each year, count the number of movies in that year that had only female actors.


In [0]:
res1 = pd.read_sql_query( 
                          "SELECT SUBSTR(M.YEAR,-4) AS YEAR\
                                  ,COUNT(DISTINCT M.MID) As Movie_F \
                          FROM MOVIE M \
                          WHERE M.MID NOT IN (\
                                              SELECT M.MID \
                                              FROM MOVIE M \
                                                  , M_Cast As C\
                                                  , Person As P \
                                                WHERE M.MID = C.MID \
                                                AND P.PID = TRIM(C.PID)\
                                                AND P.Gender ='Male' \
                                              )\
                          GROUP BY SUBSTR(M.YEAR,-4)\
                  ", conn) 
res1

,YEAR,Movie_F
0,1939,1
1,1999,1
2,2000,1
3,2009,1
4,2012,1
5,2018,2


b. Now include a small change: report for each year the percentage of movies in that
year with only female actors, and the total number of movies made that year. For
example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were
13,522 movies, and 31.81% had only female actors. You do not need to round your

answer

In [0]:
res1 = pd.read_sql_query( 
                        "SELECT SUBSTR(M.YEAR,-4) AS YEAR\
                                ,COUNT(DISTINCT M.MID) As Movie_T \
                                ,CAST( (T1.MOVIE_F * 100.00) / COUNT(DISTINCT M.MID) AS FLOAT)  AS PER_F \
                        FROM MOVIE M \
                        ,                   (\
                                          SELECT SUBSTR(M.YEAR,-4) AS YEAR\
                                                  ,COUNT(DISTINCT M.MID) As Movie_F \
                                          FROM MOVIE M \
                                          WHERE M.MID NOT IN (\
                                                              SELECT M.MID \
                                                              FROM MOVIE M \
                                                                  , M_Cast As C\
                                                                  , Person As P \
                                                                WHERE M.MID = C.MID \
                                                                AND P.PID = TRIM(C.PID)\
                                                                AND P.Gender ='Male' \
                                                              )\
                                          GROUP BY SUBSTR(M.YEAR,-4)\
                                          ) AS T1 \
                          WHERE CAST(SUBSTR(M.YEAR,-4) AS INT) = T1.YEAR \
                          GROUP BY SUBSTR(M.YEAR,-4)\
                    ", conn) 
res1

,YEAR,Movie_T,PER_F
0,1939,2,50.000000
1,1999,66,1.515152
2,2000,64,1.562500
3,2009,110,0.909091
4,2012,111,0.900901
5,2018,104,1.923077


6. Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast
size" we mean the number of distinct actors that played in that movie: if an actor played
multiple roles, or if it simply occurs multiple times in casts, we still count her/him only
once.

In [0]:
res1 = pd.read_sql_query("SELECT\
                            J1.MOVIE_TITLE \
                            ,MAX(J1.CAST_SIZE) AS CAST_SIZE \
                            FROM \
                            (SELECT  \
                            COUNT(DISTINCT C.PID) As CAST_SIZE \
                            ,TRIM(M.TITLE) AS MOVIE_TITLE \
                            FROM Movie As M \
                            LEFT JOIN M_Cast As C \
                            ON M.MID = C.MID \
                            GROUP BY TRIM(M.TITLE) ) J1\
     ", conn)
res1

,MOVIE_TITLE,CAST_SIZE
0,Ocean's Eight,238


7. A decade is a sequence of 10 consecutive years. For example, say in your database you have
movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the
second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of
films and the total number of films in D.

In [0]:
res1 = pd.read_sql_query(" \
    SELECT Q1.DECADE \
    ,MAX(Q1.Movie_ount) As Total_No_Films\
    FROM \
    (SELECT \
    COUNT(DISTINCT M.MID) AS Movie_ount\
    , '1965 to 1974' AS DECADE \
    FROM MOVIE AS M \
     WHERE  CAST(SUBSTR(M.YEAR,-4) AS INT) BETWEEN 1965 AND 1974 \
     UNION \
     SELECT \
     COUNT(DISTINCT M.MID) AS Movie_ount\
    , '1975 to 1984' AS DECADE \
    FROM MOVIE AS M \
     WHERE  CAST(SUBSTR(M.YEAR,-4) AS INT) BETWEEN 1975 AND 1984 \
     UNION \
     SELECT \
     COUNT(DISTINCT M.MID) AS Movie_ount\
    , '1985 to 1994' AS DECADE \
    FROM MOVIE AS M \
     WHERE  CAST(SUBSTR(M.YEAR,-4) AS INT) BETWEEN 1985 AND 1994 \
     UNION \
     SELECT \
     COUNT(DISTINCT M.MID) AS Movie_ount\
    , '1995 to 2004' AS DECADE \
    FROM MOVIE AS M \
     WHERE  CAST(SUBSTR(M.YEAR,-4) AS INT) BETWEEN 1995 AND 2004 \
     UNION \
     SELECT \
     COUNT(DISTINCT M.MID) AS Movie_ount\
    , '2005 to 2014' AS DECADE \
    FROM MOVIE AS M \
     WHERE  CAST(SUBSTR(M.YEAR,-4) AS INT) BETWEEN 2005 AND 2014 \
     ) Q1\
        ", conn)
res1

,DECADE,Total_No_Films
0,2005 to 2014,1170


8. Find the actors that were never unemployed for more than 3 years at a stretch. (Assume
that the actors remain unemployed between two consecutive movies).


In [0]:
res1 = pd.read_sql_query(
                          "SELECT TRIM(P.NAME) AS ACTOR_NAME \
                          FROM Movie As M \
                          ,   Person As P \
                          ,         M_Cast As C \
                          WHERE M.MID = C.MID \
                          AND P.PID = TRIM(C.PID) \
                          AND P.PID NOT IN \
                          (SELECT P1.PID \
                                FROM Movie As M1 \
                                , Person As P1 \
                                , M_Cast As C  \
                                , Movie  As M2 \
                                , M_Cast As C2 \
                            WHERE M1.MID = C.MID \
                            AND M2.MID = C2.MID \
                            AND TRIM(C.PID) = TRIM(C2.PID) \
                            AND P1.PID = TRIM(C.PID) \
                            AND M1.YEAR - M2.YEAR > 3 \
                            ) \
      ", conn)
res1

,ACTOR_NAME
0,Christian Bale
1,Cate Blanchett
2,John Benfield
3,Lorna Brown
4,Patrick Godfrey
...,...
29446,Hayley Cleghorn
29447,Nirvasha Jithoo
29448,Jaipreet Nagra
29449,Iqbal


9. Find all the actors that made more movies with Yash Chopra than any other director.

In [11]:
res1 = pd.read_sql_query(
            "SELECT DISTINCT T1.ACTOR_NAME FROM \
                              (SELECT P1.NAME AS ACTOR_NAME \
                              ,COUNT(M.MID) AS MOVIE_COUNT_Y \
                              ,P1.PID AS PID \
                              FROM Movie As M \
                              LEFT JOIN Person As P1 \
                              ,         Person As P2 \
                              ,         M_Cast As C \
                              ,         M_Director MD \
                              ON  M.MID = C.MID \
                              AND P1.PID = TRIM(C.PID) \
                              AND M.MID = MD.MID \
                              AND P2.PID = MD.PID \
                              WHERE  TRIM(P2.NAME) LIKE 'Yash Chopra' \
                              GROUP BY TRIM(P1.Name), P1.PID \
                              ) T1 \
                      JOIN \
                            (SELECT P1.NAME AS ACTOR_NAME \
                              ,P2.NAME AS DIRECTOR_NAME \
                              ,COUNT(M.MID) AS MOVIE_COUNT \
                              ,P1.PID AS PID \
                              FROM Movie As M \
                              LEFT JOIN Person As P1 \
                              ,         Person As P2 \
                              ,         M_Cast As C \
                              ,         M_Director MD \
                              ON  M.MID = C.MID \
                              AND P1.PID = TRIM(C.PID) \
                              AND M.MID = MD.MID \
                              AND P2.PID = MD.PID \
                              WHERE  TRIM(P2.NAME) NOT LIKE 'Yash Chopra' \
                              GROUP BY P1.Name, P2.Name, P1.PID \
                            ) T2 \
            ON T1.PID = T2.PID \
            AND T1.MOVIE_COUNT_Y  > T2.MOVIE_COUNT    ", conn)
res1

,ACTOR_NAME
0,A.K. Hangal
1,Aamir Khan
2,Achala Sachdev
3,Ajit
4,Akhilendra Mishra
...,...
335,Waheeda Rehman
336,Yash Chopra
337,Yashodra Katju
338,Yunus Parvez


10. The Shahrukh number of an actor is the length of the shortest path between the actor and
Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all
actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who
acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc.
Return all actors whose Shahrukh number is 2.

In [5]:
res1 = pd.read_sql_query("\
                          SELECT DISTINCT P1.NAME \
                          FROM MOVIE AS M1 \
                          LEFT JOIN Person As P1 \
                          ,         M_Cast As C1 \
                          ON  M1.MID = C1.MID \
                          AND P1.PID = TRIM(C1.PID) \
                          AND M1.MID IN(\
                                        SELECT DISTINCT M1.MID \
                                        FROM MOVIE AS M1 \
                                        LEFT JOIN Person As P1 \
                                        ,         M_Cast As C1 \
                                        ON  M1.MID = C1.MID \
                                        AND P1.PID = TRIM(C1.PID) \
                                        AND P1.PID IN\
                                                      (\
                                                      SELECT DISTINCT P1.PID \
                                                      FROM MOVIE AS M1 \
                                                      LEFT JOIN Person As P1 \
                                                      ,         M_Cast As C1 \
                                                      ON  M1.MID = C1.MID \
                                                      AND P1.PID = TRIM(C1.PID) \
                                                      AND P1.PID NOT IN('nm0451321')\
                                                      WHERE M1.MID IN (\
                                                                      SELECT DISTINCT M1.MID \
                                                                      FROM MOVIE AS M1 \
                                                                      LEFT JOIN Person As P1 \
                                                                      ,         M_Cast As C1 \
                                                                      ON  M1.MID = C1.MID \
                                                                      AND P1.PID = TRIM(C1.PID)\
                                                                      WHERE TRIM(P1.NAME) LIKE '%Shah Rukh Khan%' \
                                                                        )\
                                                      )\
                                  )\
                                          ", conn)
res1

,Name
0,Alicia Vikander
1,Dominic West
2,Walton Goggins
3,Daniel Wu
4,Kristin Scott Thomas
...,...
26877,Minoo Mehtab
26878,Hayley Cleghorn
26879,Nirvasha Jithoo
26880,Kamal Maharshi
